In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4044
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1641367542655)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6efbab17


집게(aggregation)는 무언가를 함께 모으는 행위이며 빅데이터 분석의 초석임<br/>
집계를 수행하려면 **키**나 **그룹**을 지정하고 하나 이상의 컬럼을 변환하는 방법을 지정하는 **집계 함수**를 사용함<br/>
*집계 함수는 여러 입력값이 주어지면 그룹별로 결과를 생성함*<br/>
일반적으로 특정 그룹의 평균값을 구하는 것과 같은 수치형 데이터 요약에 집계를 사용할 수 있음<br/>
이런 연산은 합산, 곱셈 또는 카운팅(counting) 등의 작업이 될 수 있음<br/>
또한 배열, 리스트 또는 맵 같은 복합 데이터 타입을 사용해 집계를 사용할 수도 있음<br/>
이와 관련된 내용은 7.1.12절 '복합 데이터 타입의 집계'에서 자세히 알아보겠음<br/>

스파크는 모든 데이터 타입을 다루는 것 외에도 다음과 같은 그룹화 데이터 타입을 생성할 수 있음<br/>
* 가장 간단한 형태의 그룹화는 select 구문에서 집계를 수행해 DataFrame 전체 데이터를 요약하는 것임
* 'group by'는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있음
* '윈도우(window)'는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있음. 하지만 함수의 입력으로 사용할 로우는 현재 로우와 어느 정도 연관성이 있어야 함
* '그룹화 셋(grouping set)'은 서로 다른 레벨의 값을 집계할 때 사용함. SQL, DataFrame의 롤업 그리고 큐브를 사용할 수 있음
* '롤업(rollup)'은 하나 이상의 키를 지정할 수 있음. 그리고 컬럼을 변환하는 데 다른 집계 함수를 사용하여 계층적으로 요약된 값을 구할 수 있음
* '큐브(cube)'는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있음. 큐브는 모든 컬럼 조합에 대한 요약값을 계산함

구매 이력 데이터를 사용해 파티션을 훨씬 적은 수로 분할할 수 있도록 repartitioning하고 빠르게 접근할 수 있도록 캐싱하겠음<br/>
파티션 수를 줄이는 이유는 적은 양의 데이터를 가진 수많은 파일이 존재하기 때문<br/>

In [2]:
val df = spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("Downloads/Spark-The-Definitive-Guide/data/retail-data/all/*.csv")
    .coalesce(5)

df.cache()
df.createOrReplaceTempView("dfTable")
spark.sql("SELECT * FROM dfTable").show(false)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |22752    |SET 7 BABUSHKA NESTING BOXES       

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [InvoiceNo: string, StockCode: string ... 6 more fields]


DataFrame을 사용해 기본 집계를 수행해보겠음<br/>
다음은 count 메서드를 사용한 간단한 예제임<br/>

In [3]:
df.count() == 541909

res2: Boolean = true


이 책을 앞에서부터 읽었다면 count 메서드가 transformation이 아닌 action이라는 사실을 알고 있을 것임<br/>
그러므로 결과를 즉시 반환함<br/>
위 예제처럼 count 메서드는 데이터셋의 전체 크기를 알아보는 용도로 사용하지만 메모리에 DataFrame 캐싱 작업을 수행하는 용도로 사용되기도 함<br/>

지금은 count 메서드가 약간 이질적으로 보일 수 있음<br/>
그 이유는 함수가 아니라 메서드 형태로 존재하고 transformation처럼 지연 연산 방식이 아닌 즉시 연산을 수행하기 때문임<br/>
다음 절에서는 지연 연산 방식으로 count 메서드를 사용하는 방법을 알아보겠음<br/>

# 7.1 집계 함수

모든 집계는 6장에서 사용한 DataFrame의 .stat 속성을 이용하는 특별한 경우를 제외하면 함수를 사용함<br/>
집계 함수는 org.apache.spark.sql.functions 패키지에서 찾아볼 수 있음<br/>

*NOTE*<br/>
스칼라와 파이썬에서 import할 수 있는 함수와 SQL에서 사용 가능한 함수 사이에는 약간의 차이가 있음<br/>
이 내용은 매 릴리스마다 변하므로 최종 목록을 이 책에 포함하는 것은 불가능하며 이 절에서는 자주 사용하는 함수 위주로 알아보겠음<br/>

## 7.1.1 count

가장 먼저 count 함수를 알아보겠음<br/>
다음 예제에서 count 함수는 action이 아닌 transformation으로 동작함<br/>
count 함수는 두 가지 방식으로 사용할 수 있음<br/>
하나는 count 함수에 특정 컬럼을 지정하는 방식이고, 다른 하나는 count(\*)나 count(1)를 사용하는 방식임<br/>
count 함수를 사용해 다음 예제와 같이 전체 로우 수를 카운트할 수 있음<br/>

In [4]:
import org.apache.spark.sql.functions.count

df.select(count("StockCode")).show( )

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



import org.apache.spark.sql.functions.count


*CAUTION*<br/>
null 값이 포함된 데이터의 레코드 수를 구할 때는 몇 가지 유의사항이 있음<br/>
예를 들어 count(\*) 구문을 사용하면 null 값을 가진 로우를 포함해 카운트함<br/>
하지만 count 함수에 특정 컬럼을 지정하면 null 값을 카운트하지 않음<br/>

## 7.1.2 countDistinct

전체 레코드 수가 아닌 고유 레코드 수를 구해야 한다면 countDistinct 함수를 사용함<br/>

In [5]:
import org.apache.spark.sql.functions.countDistinct

df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



import org.apache.spark.sql.functions.countDistinct


## 7.1.3 approx_count_distinct

대규모 데이터셋을 다루다 보면 정확한 고유 개수가 무의미할 때도 있음<br/>
어느 정도 수준의 정확도를 가지는 근사치만으로 유의미하다면 approx_count_distinct 함수를 사용해 근사치를 계산할 수 있음<br/>

In [6]:
import org.apache.spark.sql.functions.approx_count_distinct

df.select(approx_count_distinct("StockCode", 0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



import org.apache.spark.sql.functions.approx_count_distinct


*의문사항*<br/>
책의 설명을 보면 근사치를 구하는 경우 더 적은 시간이 걸린다고 되어 있는데 이 예제에서 count_distinct와 approx_count_distinct의 실행 시간 차이가 궁금해짐<br/>
따라서 다음의 코드로 실행시간을 비교해봤음<br/>

In [12]:
import org.apache.spark.sql.functions.{countDistinct, approx_count_distinct}

val startTimeCountDistinct = System.nanoTime
df.select(countDistinct("StockCode"))
val durationCountDistinct = (System.nanoTime - startTimeCountDistinct)
println(s"dutation time for countDistinct = $durationCountDistinct")

val startTimeApproxCountDistinct = System.nanoTime
df.select(approx_count_distinct("StockCode", 0.1))
val durationApproxCountDistinct = (System.nanoTime - startTimeApproxCountDistinct)
println(s"dutation time for approx_count_distinct = $durationApproxCountDistinct")

dutation time for countDistinct = 11807765
dutation time for approx_count_distinct = 11002001


import org.apache.spark.sql.functions.{countDistinct, approx_count_distinct}
startTimeCountDistinct: Long = 777515063632166
durationCountDistinct: Long = 11807765
startTimeApproxCountDistinct: Long = 777515075536696
durationApproxCountDistinct: Long = 11002001


몇 차례 확인해본 결과 큰 차이 없이 countDistinct가 더 적은 시간이 걸리는 경우도, approx_count_distinct가 더 적은 시간이 걸리는 경우도 있었음<br/>
아마 훨씬 더 큰 데이터를 가지고 실험해본다면 더 유의미한 차이를 확인할 수 있지 않을까 싶음<br/>

approx_count_distinct 함수는 최대 추정 오류율(maximum estimation error)이라는 한 가지 파라미터를 사용함<br/>
예제에서는 큰 오류율을 설정했기 때문에 기대치에서 크게 벗어나는 결과를 얻게 되지만, countDistinct 함수보다 더 빠르게 결과를 반환함<br/>
이 함수의 성능은 대규모 데이터셋을 사용할 때 훨씬 더 좋아짐<br/>

## 7.1.4 first와 last

함수명에서도 알 수 있듯이 first와 last 함수는 DataFrame의 첫 번째 값이나 마지막 값을 얻을 때 사용함<br/>
이들 함수는 DataFrame의 값이 아닌 로우를 기반으로 동작함<br/>

In [13]:
import org.apache.spark.sql.functions.{first, last}

df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



import org.apache.spark.sql.functions.{first, last}


## 7.1.5 min과 max

DataFrame에서 최솟값과 최댓값을 추출하려면 min과 max 함수를 사용함<br/>

In [14]:
import org.apache.spark.sql.functions.{min, max}

df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



import org.apache.spark.sql.functions.{min, max}


## 7.1.6 sum

DataFrame에서 특정 컬럼의 모든 값을 합산하려면 sum 함수를 사용함<br/>

In [15]:
import org.apache.spark.sql.functions.sum

df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



import org.apache.spark.sql.functions.sum


## 7.1.7 sumDistinct

특정 컬럼의 모든 값을 합산하는 방법 외에도 sumDistinct 함수를 사용해 고윳값을 합산할 수 있음

In [16]:
import org.apache.spark.sql.functions.sumDistinct

df.select(sumDistinct("Quantity")).show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



import org.apache.spark.sql.functions.sumDistinct


## 7.1.8 avg

sum 함수의 결과를 count 함수의 결과로 나누어 평균값을 구할 수 있지만, 스파크의 avg 함수나 mean 함수를 사용하면 평균값을 더 쉽게 구할 수 있음<br/>
다음 예제는 집계된 컬럼을 재활용하기 위해 alias 메서드를 사용함<br/>

In [17]:
import org.apache.spark.sql.functions.{sum, count, avg, expr}

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))
    .selectExpr(
        "total_purchases/total_transactions",
        "avg_purchases",
        "mean_purchases").show(false)

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|avg_purchases   |mean_purchases  |
+--------------------------------------+----------------+----------------+
|9.55224954743324                      |9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



import org.apache.spark.sql.functions.{sum, count, avg, expr}


*NOTE*<br/>
distinct 함수를 사용해 모든 고윳값의 평균을 구할 수도 있음<br/>
대부분의 집계 함수는 고윳값을 사용해 집계를 수행하는 방식을 지원함<br/>

## 7.1.9 분산과 표준편차

평균을 구하다 보면 자연스럽게 분산과 표준편차가 궁금해짐<br/>
분산과 표준편차는 평균 주변에 데이터가 분포된 정도를 측정하는 방법임<br/>
분산은 평균과의 차이를 제곱한 결과의 평균이며 표준편차는 분산의 제곱근임<br/>
스파크에서는 함수를 사용해 분산과 표준편차를 계산할 수 있음<br/>
스파크는 표본표준편차(sample standard deviation)뿐만 아니라 모표준편차(population standard deviation) 방식도 지원하므로 주의가 필요함<br/>

variance 함수나 stddev 함수를 사용한다면 기본적으로 표본표준분산(sample standard variance)과 표본표준편차 공식을 이용함<br/>

모표준분산(population standard variance)이나 모표준편차 방식을 사용하려면 다음 예제와 같이 var_pop 함수나 stddev_pop 함수를 사용함<br/>

In [19]:
import org.apache.spark.sql.functions.{var_pop, stddev_pop}
import org.apache.spark.sql.functions.{var_samp, stddev_samp}

df.select(var_pop("Quantity"), var_samp("Quantity"),
         stddev_pop("Quantity"), stddev_samp("Quantity")).show(false)

+------------------+------------------+--------------------+---------------------+
|var_pop(Quantity) |var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609056|47559.391409298754|218.08095663447796  |218.08115785023418   |
+------------------+------------------+--------------------+---------------------+



import org.apache.spark.sql.functions.{var_pop, stddev_pop}
import org.apache.spark.sql.functions.{var_samp, stddev_samp}


## 7.1.10 비대칭도와 첨도

비대칭도(skewness)와 첨도(kurtosis) 모두 데이터의 변곡점(extreme point)을 측정하는 방법임<br/>
비대칭도는 데이터 평균의 비대칭 정도를 측정하고, 첨도는 데이터 끝 부분을 측정함<br/>
비대칭도와 첨도는 확률변수(random variable)의 확률분포(probability distribution)로 데이터를 모델링할 때 특히 중요함<br/>
비록 함수를 사용해 비대칭도와 첨도를 계산해보겠음<br/>

In [20]:
import org.apache.spark.sql.functions.{skewness, kurtosis}

df.select(skewness("Quantity"), kurtosis("Quantity")).show(false)

+-------------------+------------------+
|skewness(Quantity) |kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052562|119768.05495536952|
+-------------------+------------------+



import org.apache.spark.sql.functions.{skewness, kurtosis}


## 7.1.11 공분산과 상관관계

지금까지 단일 컬럼 집계 함수를 알아보았음<br/>
이제 두 컬럼값 사이의 영향도를 비교하는 함수를 알아보겠음<br/>
cov와 corr 함수를 사용해 공분산(covariance)과 상관관계(correlation)를 계산할 수 있음<br/>
공분산은 데이터 입력값에 따라 다른 범위를 가짐<br/>
상관관계는 피어슨 상관계수(Pearson correlation coefficient)를 측정하며 -1과 1 사이의 값을 가짐<br/>

var 함수처럼 표본공분산(sample covariance) 방식이나 모공분산(population covariance) 방식으로 공분산을 계산할 수도 있음<br/>
그러므로 사용하고자 하는 방식을 명확하게 지정하는 것이 좋음<br/>
상관관계는 모집단이나 표본에 대한 계산 개념이 없음<br/>
공분산과 상관관계에 대한 계산을 수행하는 방법은 다음과 같음<br/>

In [21]:
import org.apache.spark.sql.functions.{corr, covar_pop, covar_samp}

df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
         covar_pop("InvoiceNo", "Quantity")).show(false)

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|4.912186085635685E-4     |1052.7280543902734             |1052.7260778741693            |
+-------------------------+-------------------------------+------------------------------+



import org.apache.spark.sql.functions.{corr, covar_pop, covar_samp}


## 7.1.12 복합 데이터 타입의 집계

스파크는 수식을 이용한 집계뿐만 아니라 복합 데이터 타입을 사용해 집계를 수행할 수 있음<br/>
예를 들어 특정 컬럼의 값을 리스트로 수집하거나 셋 데이터 타입으로 고윳값만 수집할 수 있음<br/>

수집된 데이터는 처리 파이프라인에서 다양한 프로그래밍 방식으로 다루거나 사용자 정의 함수를 사용해 전체 데이터에 접근할 수 있음<br/>

In [25]:
import org.apache.spark.sql.functions.{collect_set, collect_list}

df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



import org.apache.spark.sql.functions.{collect_set, collect_list}


# 7.2 그룹화

지금까지 DataFrame 수준의 집계만 다뤘음<br/>
하지만 데이터 **그룹** 기반의 집계를 수행하는 경우가 더 많음<br/>
데이터 그룹 기반의 집계는 단일 컬럼의 데이터를 그룹화하고 해당 그룹의 다른 여러 컬럼을 사용해서 계산하기 위해 카테고리형 데이터(categorical data)를 사용함<br/>

데이터 그룹 기반의 집계를 설명하는 데 가장 좋은 방법은 그룹화를 해보는 것임<br/>
이전에 했던 것처럼 count를 가장 먼저 수행함<br/>
고유한 송장번호(InvoiceNo)를 기준으로 그룹을 만들고 그룹별 물품 수를 count함<br/>
이 연산은 또 다른 DataFrame을 변환하며 지연 처리 방식으로 수행됨<br/>

그룹화 작업은 하나 이상의 컬럼을 그룹화하고 집계 연산을 수행하는 두 단계로 이뤄짐<br/>
첫 번째 단계에서는 RelationalGroupedDataset이 반환되고, 두 번째 단계에서는 DataFrame이 반환됨<br/>

그룹의 기준이 되는 컬럼을 여러 개 지정할 수 있음<br/>

In [28]:
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   536596|      null|    6|
|   537252|      null|    1|
|   538041|      null|    1|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
+---------+----------+-----+
only showing top 20 rows



## 7.2.1 표현식을 이용한 그룹화

카운팅은 메서드로 사용할 수 있으므로 특별함<br/>
*하지만 메서드 대신 count 함수를 사용할 것을 추천함*<br/>
*또한 count 함수를 select 구문에 표현식으로 지정하는 것보다 agg 메서드를 사용하는 것이 좋음*<br/>
agg 메서드는 여러 집계 처리를 한 번에 지정할 수 있으며, 집계에 표현식을 사용할 수 있음<br/>
또한 transformation이 완료된 컬럼에 alias 메서드를 사용할 수 있음<br/>

In [32]:
import org.apache.spark.sql.functions.count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)"))
    .show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows



import org.apache.spark.sql.functions.count


## 7.2.2 맵을 이용한 그룹화

컬럼을 키로, 수행할 집계 함수의 문자열을 값으로 하는 맵(map) 타입을 사용해 transformation을 정의할 수 있음<br/>
수행할 집계 함수를 한 줄로 작성하면 여러 컬럼명을 재사용할 수 있음<br/>

In [34]:
df.groupBy("InvoiceNo").agg("Quantity" -> "avg", "Quantity" -> "stddev_pop", "Quantity" -> "var_pop").show()

+---------+------------------+--------------------+------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)| var_pop(Quantity)|
+---------+------------------+--------------------+------------------+
|   536596|               1.5|  1.1180339887498947|1.2499999999999998|
|   536938|33.142857142857146|  20.698023172885524|428.40816326530614|
|   537252|              31.0|                 0.0|               0.0|
|   537691|              8.15|   5.597097462078001|31.327500000000004|
|   538041|              30.0|                 0.0|               0.0|
|   538184|12.076923076923077|   8.142590198943392| 66.30177514792898|
|   538517|3.0377358490566038|  2.3946659604837897|  5.73442506229975|
|   538879|21.157894736842106|  11.811070444356483|139.50138504155126|
|   539275|              26.0|  12.806248474865697|             164.0|
|   539630|20.333333333333332|  10.225241100118645|104.55555555555556|
|   540499|              3.75|  2.6653642652865788| 7.104166666666664|
|   54

# 7.3 윈도우 함수

**윈도우 함수**를 집계에 사용할 수도 있음<br/>
윈도우 함수는 데이터의 특정 '윈도우(window)'를 대상으로 고유의 집계 연산을 수행함<br/>
데이터의 '윈도우'는 현재 데이터에 대한 참조(reference)를 사용해 정의함<br/>
윈도우 명세(window specification)는 함수에 전달될 로우를 결정함<br/>
표준 group-by 함수와 유사해보일 수도 있으므로 이 둘의 차이점을 알아보겠음<br/>

*표준 group-by 함수와 윈도우 함수를 집계에 사용하는 경우의 차이?*<br/>
group-by 함수를 사용하면 모든 로우 레코드가 단일 그룹으로만 이동함<br/>
윈도우 함수는 프레임(frame)에 입력되는 모든 로우에 대해 결과값을 계산함<br/>
프레임은 로우 그룹 기반의 테이블을 의미함<br/>
각 로우는 하나 이상의 프레임에 할당될 수 있음<br/>
가장 흔하게 사용되는 방법 중 하나는 하루를 나타내는 값의 rolling average를 구하는 것임<br/>
이 작업을 수행하려면 로우가 7개의 다른 프레임으로 구성되어야 함<br/>
프레임을 정의하는 방법은 잠시 후에 알아보겠음<br/>

참고로 스파크는 다음 3가지 종류의 윈도우 함수를 지원함<br/>
* 랭크 함수 (ranking function)
* 분석 함수 (analytic function)
* 집계 함수 (aggregate function)

예제를 위해 주문 일자(InvoiceDate) 컬럼을 변환해 'date' 컬럼을 만듦<br/>
이 컬럼은 시간 정보를 제외한 날짜 정보만 가짐<br/>

In [43]:
import org.apache.spark.sql.functions.{col, to_date}

val dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")
spark.sql("SELECT * FROM dfWithDate").show(false)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |date      |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+----------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|2010-12-01|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|2010-12-01|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850 

import org.apache.spark.sql.functions.{col, to_date}
dfWithDate: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


윈도우 함수를 정의하기 위해 첫 번째 단계로 윈도우 명세를 만듦<br/>
여기서 사용하는 partitionBy 메서드는 지금까지 사용해온 파티셔닝 스키마의 개념과는 관련이 없으며 그룹을 어떻게 나눌지 결정하는 것과 유사한 개념임<br/>
orderBy 메서드는 파티션의 정렬 방식을 정의함<br/>
그리고 프레임 명세(rowsBetween 구문)는 입력된 로우의 참조를 기반으로 프레임에 로우가 포함될 수 있는지 결정함<br/>
다음 예제에서는 첫 로우부터 현재 로우까지 확인함<br/>

In [48]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.col

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY") // 예제가 스파크 2.x 기준이므로 여기(3.1.2)에서는 이 코드 없이 밑의 코드들 실행하다 보면 파싱 에러가 남

val windowSpec = Window
    .partitionBy("CustomerId", "date")
    .orderBy(col("Quantity").desc)
    .rowsBetween(Window.unboundedPreceding, Window.currentRow) // 프레임 명세(rowsBetween 구문)는 입력된 로우의 참조를 기반으로 프레임에 로우가 포함될 수 있는지 결정함

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.col
windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3e778bfa


이제 집계 함수를 사용해 고객을 좀 더 자세히 살펴보겠음<br/>
여기서는 시간대별 최대 구매 개수를 구하는 예를 들어보겠음<br/>
그러려면 위 예제에서 사용한 집계 함수에 컬럼명이나 표현식을 전달해야 함<br/>
그리고 이 함수를 적용할 데이터 프레임이 정의된 윈도우 명세도 함께 사용함<br/>

In [49]:
import org.apache.spark.sql.functions.max

val maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

import org.apache.spark.sql.functions.max
maxPurchaseQuantity: org.apache.spark.sql.Column = max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)


앞 예제는 컬럼이나 표현식을 반환하므로 DataFrame의 select 구문에서 사용할 수 있음<br/>
먼저 구매량 순위를 만들겠음<br/>
dense_rank 함수를 사용해 모든 고객에 대해 최대 구매 수량을 가진 날짜가 언제인지 알아보겠음<br/>
동일한 값이 나오거나 중복 로우가 발생해 순위가 비어 있을 수 있으므로 rank 함수 대신 dense_rank 함수를 사용함<br/>

In [50]:
import org.apache.spark.sql.functions.{dense_rank, rank}

val purchaseDenseRank = dense_rank().over(windowSpec)
val purchaseRank = rank().over(windowSpec)

import org.apache.spark.sql.functions.{dense_rank, rank}
purchaseDenseRank: org.apache.spark.sql.Column = DENSE_RANK() OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
purchaseRank: org.apache.spark.sql.Column = RANK() OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)


이 예제 또한 select 구문에서 사용할 수 있는 컬럼을 반환함<br/>
이제 select 메서드를 사용해 계산된 윈도우값을 확인해보겠음<br/>

In [51]:
import org.apache.spark.sql.functions.col

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")
    .select(
        col("CustomerId"),
        col("date"),
        col("Quantity"),
        purchaseRank.alias("quantityRank"),
        purchaseDenseRank.alias("quantityDenseRank"),
        maxPurchaseQuantity.alias("maxPurchaseQuantity"))
        .show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

import org.apache.spark.sql.functions.col


# 7.4 그룹화 셋

지금까지는 컬럼의 값을 이용해 여러 컬럼을 집계하는 데 group-by 표현식을 사용했음<br/>
때로는 여러 그룹에 걸쳐 집계할 수 있는 무언가가 필요할 수 있는데, 이 때 사용할 수 있는 것이 **그룹화 셋**임<br/>
그룹화 셋은 여러 집계를 결합하는 저수준 기능임<br/>
그룹화 셋을 이용하면 group-by 구문에서 원하는 형태로 집계를 생성할 수 있음<br/>

이해를 돕기 위해 다음 예제를 살펴보겠음<br/>
재고 코드(stockCode)와 고객(CustomerId)별 총 수량을 얻기 위해 다음과 같은 SQL 표현식을 사용함<br/>

In [68]:
val dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

dfNoNull: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


In [ ]:
/*
-- SQL

SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY CustomerId, stockCode
ORDER BY CustomerId DESC, stockCode DESC

*/

그룹화 셋을 사용해 동일한 작업을 수행할 수 있음<br/>

In [ ]:
/*
--SQL

SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY CustomerId, stockCode GROUPING SETS((CustomerId, stockCode))
ORDER BY CustomerId DESC, stockCode DESC
*/

*GROUPING SETS 구문은 SQL에서만 사용할 수 있음*<br/>
DataFrame에서 동일한 연산을 수행하려면 rollup 메서드와 cube 메서드를 사용함<br/>
롤업과 큐브에 대해 알아보겠음<br/>

## 7.4.1 롤업

지금까지는 명시적 그룹화에 대해 알아보았음<br/>
다양한 컬럼을 그룹화 키로 설정하면 그룹화 키로 설정된 조합뿐만 아니라 데이터셋에서 볼 수 있는 실제 조합을 모두 살펴볼 수 있음<br/>
롤업은 group-by 스타일의 다양한 연산을 수행할 수 있는 다차원 집계 기능임<br/>

다음 예제에서는 시간(신규 Date 컬럼)을 축으로 하는 롤업을 생성함<br/>
롤업의 결과로 생성된 DataFrame은 모든 날짜의 총합, 날짜별 총합, 날짜별 국가별 총합을 포함함<br/>

In [69]:
val rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))
    .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")
    .orderBy("Date")
rolledUpDF.show(false)

+----------+--------------+--------------+
|Date      |Country       |total_quantity|
+----------+--------------+--------------+
|null      |null          |5176450       |
|2010-12-01|EIRE          |243           |
|2010-12-01|Norway        |1852          |
|2010-12-01|Germany       |117           |
|2010-12-01|France        |449           |
|2010-12-01|null          |26814         |
|2010-12-01|United Kingdom|23949         |
|2010-12-01|Australia     |107           |
|2010-12-01|Netherlands   |97            |
|2010-12-02|EIRE          |4             |
|2010-12-02|null          |21023         |
|2010-12-02|Germany       |146           |
|2010-12-02|United Kingdom|20873         |
|2010-12-03|null          |14830         |
|2010-12-03|EIRE          |2575          |
|2010-12-03|Germany       |170           |
|2010-12-03|Switzerland   |110           |
|2010-12-03|Portugal      |65            |
|2010-12-03|Poland        |140           |
|2010-12-03|France        |239           |
+----------

rolledUpDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: date, Country: string ... 1 more field]


null 값을 가진 로우에서 전체 날짜의 합계를 확인할 수 있음<br/>
롤업된 두 개의 컬럼값이 모두 null인 로우는 두 컬럼에 속한 레코드의 전체 합계를 나타냄<br/>

In [70]:
rolledUpDF.where("Country IS NULL").show(false)

+----------+-------+--------------+
|Date      |Country|total_quantity|
+----------+-------+--------------+
|null      |null   |5176450       |
|2010-12-01|null   |26814         |
|2010-12-02|null   |21023         |
|2010-12-03|null   |14830         |
|2010-12-05|null   |16395         |
|2010-12-06|null   |21419         |
|2010-12-07|null   |24995         |
|2010-12-08|null   |22741         |
|2010-12-09|null   |18431         |
|2010-12-10|null   |20297         |
|2010-12-12|null   |10565         |
|2010-12-13|null   |17623         |
|2010-12-14|null   |20098         |
|2010-12-15|null   |18229         |
|2010-12-16|null   |29632         |
|2010-12-17|null   |16069         |
|2010-12-19|null   |3795          |
|2010-12-20|null   |14965         |
|2010-12-21|null   |15467         |
|2010-12-22|null   |3192          |
+----------+-------+--------------+
only showing top 20 rows



In [71]:
rolledUpDF.where("Date IS NULL").show(false)

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|null|null   |5176450       |
+----+-------+--------------+



## 7.4.2 큐브

큐브(cube)는 롤업을 고차원적으로 사용할 수 있게 해줌<br/>
큐브는 요소들을 계층적으로 다루는 대신 모든 차원에 대해 동일한 작업을 수행함<br/>
즉, 전체 기간에 대해 날짜와 국가별 결과를 얻을 수 있음<br/>
기존 기능만으로 다음과 같은 정보를 가진 테이블을 만들 수 있을까?<br/>
* 전체 날짜와 모든 국가에 대한 합계
* 모든 국가의 날짜별 합계
* 날짜별 국가별 합계
* 전체 날짜의 국가별 합계

메서드 호출 방식은 롤업과 매우 유사하며 rollup 메서드 대신 cube 메서드를 호출함<br/>

In [72]:
dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))
    .select("Date", "Country", "sum(Quantity)").orderBy("Date").show(false)

+----+--------------------+-------------+
|Date|Country             |sum(Quantity)|
+----+--------------------+-------------+
|null|Czech Republic      |592          |
|null|null                |5176450      |
|null|RSA                 |352          |
|null|Sweden              |35637        |
|null|Lebanon             |386          |
|null|Italy               |7999         |
|null|Germany             |117448       |
|null|Japan               |25218        |
|null|European Community  |497          |
|null|USA                 |1034         |
|null|Malta               |944          |
|null|Poland              |3653         |
|null|France              |110480       |
|null|United Arab Emirates|982          |
|null|Greece              |1556         |
|null|United Kingdom      |4263829      |
|null|Lithuania           |652          |
|null|Iceland             |2458         |
|null|Singapore           |5234         |
|null|Finland             |10666        |
+----+--------------------+-------

큐브를 사용해 테이블에 있는 모든 정보를 빠르고 쉽게 조회할 수 있는 요약 정보 테이블을 만들 수 있음<br/>

## 7.4.3 그룹화 메타데이터

큐브와 롤업을 사용하다 보면 집계 수준에 따라 쉽게 필터링하기 위해 집계 수준을 조회하는 경우가 발생함<br/>
이때 grouping_id를 사용함<br/>
grouping_id는 결과 데이터셋의 집계 수준을 명시하는 컬럼을 제공함<br/>
예제의 쿼리는 다음과 같은 4개의 개별 그룹화 ID 값을 반환함<br/>


그룹화 ID / 설명
* 3 : 가장 높은 계층의 집계 결과에서 나타남. customerId나 stockCode에 관계없이 총 수량을 제공함
* 2 : 개별 재고 코드의 모든 집계 결과에서 나타남. customerId에 관계없이 재고 코드별 총 수량을 제공함
* 1 : 구매한 물품에 관계없이 customerId를 기반으로 총 수량을 제공함
* 0 : customerId와 stockCode별 조합에 따라 총 수량을 제공함

In [73]:
import org.apache.spark.sql.functions.{grouping_id, sum, expr}

dfNoNull.cube("customerId", "stockCode").agg(grouping_id(), sum("Quantity"))
    .orderBy(col("grouping_id()").desc)
    .show(false)

+----------+---------+-------------+-------------+
|customerId|stockCode|grouping_id()|sum(Quantity)|
+----------+---------+-------------+-------------+
|null      |null     |3            |5176450      |
|null      |85049G   |2            |1601         |
|null      |21913    |2            |395          |
|null      |90199C   |2            |18           |
|null      |21257    |2            |590          |
|null      |22352    |2            |3077         |
|null      |22090    |2            |7516         |
|null      |85214    |2            |234          |
|null      |84884a   |2            |38           |
|null      |21054    |2            |41           |
|null      |22943    |2            |1318         |
|null      |85047    |2            |0            |
|null      |84192    |2            |115          |
|null      |22047    |2            |3400         |
|null      |84548    |2            |-36          |
|null      |23120    |2            |1644         |
|null      |72802a   |2        

import org.apache.spark.sql.functions.{grouping_id, sum, expr}


## 7.4.4 피벗

피벗(pivot)을 사용해 로우를 컬럼으로 변환할 수 있음<br/>
현재 데이터셋에는 Country 컬럼이 있음<br/>
피벗을 사용해 국가별로 집계 함수를 적용할 수 있으며 쿼리를 사용해 쉽게 결과를 확인할 수 있음<br/>

In [74]:
val pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

pivoted: org.apache.spark.sql.DataFrame = [date: date, Australia_sum(Quantity): bigint ... 113 more fields]


DataFrame은 국가명, 수치형 변수 그리고 날짜를 나타내는 컬럼을 조합한 컬럼을 가짐<br/>
예를 들어 USA와 관련된 컬럼을 살펴보면 USA_sum(Quantity), USA_sum(UnitPrice) 그리고 USA_sum(CustomerId)가 있음<br/>
또한 집계를 수행했기 때문에 수치형 컬럼으로 나타남<br/>

다음은 예제 쿼리의 결과임<br/>

In [83]:
pivoted.where("date > '2011-12-05'").select("date", "USA_sum(Quantity)", "USA_sum(UnitPrice)", "USA_sum(CustomerId)").show(10)

+----------+-----------------+------------------+-------------------+
|      date|USA_sum(Quantity)|USA_sum(UnitPrice)|USA_sum(CustomerId)|
+----------+-----------------+------------------+-------------------+
|2011-12-06|             null|              null|               null|
|2011-12-09|             null|              null|               null|
|2011-12-08|             -196|             13.75|             138138|
|2011-12-07|             null|              null|               null|
+----------+-----------------+------------------+-------------------+



이제 컬럼의 모든 값을 단일 그룹화해서 계산할 수 있음<br/>
하지만 데이터를 탐색하는 방식에 따라 피벗을 수행한 결과값이 감소할 수도 있음<br/>
특정 컬럼의 cardinality가 낮다면 스키마와 쿼리 대상을 확인할 수 있도록 피벗을 사용해 다수의 컬럼으로 변환하는 것이 좋음<br/>

# 7.5 사용자 정의 집계 함수

사용자 정의 집계 함수(user-defined aggregation function, UDAF)는 직접 제작한 함수나 비즈니스 규칙에 기반을 둔 자체 집계 함수를 정의하는 방법임<br/>
UDAF를 사용해서 입력 데이터 그룹에 직접 개발한 연산을 수행할 수 있음<br/>
스파크는 입력 데이터의 모든 그룹의 중간 결과를 단일 AggregationBuffer에 저장해 관리함<br/>

UDAF를 생성하려면 기본 클래스인 UserDefinedAggregateFunction을 상속받음<br/>
그리고 다음과 같은 메서드를 정의해야 함<br/>
* inputSchema: UDAF 입력 파라미터의 스키마를 StructType으로 정의
* bufferSchema: UDAF 중간 결과의 스키마를 StructType으로 정의
* dataType: 반환될 값의 DataType을 정의
* deterministic: UDAF가 동일한 입력값에 대해 항상 동일한 결과를 반환하는지 불리언 값으로 정으
* initialize: 집계용 버퍼의 값을 초기화하는 로직을 정의
* update: 입력받은 로우를 기반으로 내부 버퍼를 업데이트하는 로직을 정의
* merge: 두 개의 집계용 버퍼를 병합하는 로직을 정의
* evaluate: 집계의 최종 결과를 생성하는 로직을 정의

다음 예제는 입력된 모든 로우의 컬럼이 true인지 아닌지 판단하는 BoolAnd 클래스를 구현함<br/>
만약 하나의 컬럼이라도 true가 아니라면 false를 반환함<br/>

In [84]:
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._

class BoolAnd extends UserDefinedAggregateFunction {
    def inputSchema: org.apache.spark.sql.types.StructType = 
        StructType(StructField("value", BooleanType) :: Nil)
    def bufferSchema: StructType = StructType(
        StructField("result", BooleanType) :: Nil)
    def dataType: DataType = BooleanType
    def deterministic: Boolean = true
    def initialize(buffer: MutableAggregationBuffer): Unit = {
        buffer(0) = true
    }
    def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
        buffer(0) = buffer.getAs[Boolean](0) && input.getAs[Boolean](0)
    }
    def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
        buffer1(0) = buffer1.getAs[Boolean](0) && buffer2.getAs[Boolean](0)
    }
    def evaluate(buffer: Row): Any = {
        buffer(0)
    }
}

import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
defined class BoolAnd


이제 간단히 클래스를 초기화하고 함수로 등록함

In [85]:
val ba = new BoolAnd
spark.udf.register("booland", ba)

import org.apache.spark.sql.functions._

spark.range(1)
    .selectExpr("explode(array(TRUE, TRUE, TRUE)) as t")
    .selectExpr("explode(array(TRUE, FALSE, TRUE)) as f", "t")
    .select(ba(col("t")), expr("booland(f)"))
    .show(false)

+----------+----------+
|booland(t)|booland(f)|
+----------+----------+
|true      |false     |
+----------+----------+



ba: BoolAnd = BoolAnd@3571d942
import org.apache.spark.sql.functions._


# 7.6 정리

이 장에서는 스파크에서 사용 가능한 여러 유형의 집계 연산에 대해 알아보았음<br/>
그룹화, 윈도우 함수 그리고 롤업과 큐브도 함께 알아보았음<br/>
다음 장에서는 여러 데이터소스를 결합할 수 있는 조인에 대해 자세히 알아보겠음<br/>